In [1]:
import keras
import h5py 
import os
os.environ['KERAS_BACKEND']='tensorflow'
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Model
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Input, Dense, Convolution2D, Conv2DTranspose, Dropout
from keras.layers import MaxPooling2D, UpSampling2D
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
print('libs loaded')

/home/ian/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


libs loaded


In [73]:
# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, _), (X_test, Y_test) = mnist.load_data()
#plt.imshow(X_train[0])

# data pre-processing
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
print(X_train.shape)
print(X_test.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [78]:
size = 28

model = Sequential()

# encoder
model.add(Convolution2D(32, (3, 3), padding='same', activation='relu', input_shape=(size,size,1)))
model.add(MaxPooling2D((2,2)))
model.add(Convolution2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))


# (None, 7, 7, 32)

# decoder
model.add(Conv2DTranspose(32, (3, 3), padding='same', activation='relu'))
model.add(UpSampling2D((2,2)))
model.add(Conv2DTranspose(32, (3, 3), padding='same', activation='relu'))
model.add(UpSampling2D((2,2)))

model.add(Conv2DTranspose(1, (3, 3), padding='same', activation='relu'))

print (model.output_shape)

opt = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=opt,
              loss='mse',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_grads=True, write_images=True)

(None, 28, 28, 1)


In [82]:
# training
model.fit(X_train, X_train,
                epochs=10,
                batch_size=200,
                shuffle=True,
                callbacks=[tensorboard])

"""
Epoch 20/20
60000/60000 [==============================] - 7s - loss: 0.0398
"""


Epoch 1/10
60000/60000 [==============================] - 3s 42us/step - loss: 0.0028 - acc: 0.8154
Epoch 2/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0026 - acc: 0.8154
Epoch 3/10
60000/60000 [==============================] - 2s 41us/step - loss: 0.0024 - acc: 0.8154
Epoch 4/10
60000/60000 [==============================] - 2s 41us/step - loss: 0.0023 - acc: 0.8154
Epoch 5/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0022 - acc: 0.8154
Epoch 6/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0021 - acc: 0.8154
Epoch 7/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0021 - acc: 0.8154
Epoch 8/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0020 - acc: 0.8154
Epoch 9/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0019 - acc: 0.8154
Epoch 10/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0019 - acc: 0.8155

'\nEpoch 20/20\n60000/60000 [==============================] - 7s - loss: 0.0398\n'

In [83]:
# save
model.save('model.h5')   # HDF5 file, you have to pip3 install h5py if don't have it
del model  # deletes the existing model


In [ ]:
# load
model = load_model('model.h5')
X = X_test[0:1]
img = model.predict(X)
img2 = img > 0.5
print(img.shape)

plt.imshow(X.reshape(28,28), cmap='gray')
plt.show()
plt.imshow(img.reshape(28,28), cmap='gray')
plt.show()
plt.imshow(img2.reshape(28,28), cmap='gray')
plt.show()